In [1]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.0 MB/s eta 0:00:00


In [2]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2023-07-21 16:34:45--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  10.5MB/s    in 7.0s    

2023-07-21 16:34:52 (6.21 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [3]:
!gunzip reviews_Cell_Phones_and_Accessories_5.json.gz

In [4]:
import gensim
import pandas as pd

In [5]:
df = pd.read_json("/content/reviews_Cell_Phones_and_Accessories_5.json",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [6]:
df.shape

(194439, 9)

# Preprocessing & Tokenization

In [7]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [8]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [9]:
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text.head()

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

# Train the Word2Vec Model

In [11]:
model = gensim.models.Word2Vec(window=10,
                               vector_size=100,
                               min_count=2,
                               sg=1)

# Build the Vocabulory

In [12]:
model.build_vocab(review_text)

In [13]:
model.epochs

5

In [14]:
model.window

10

# Train Word2Vec Model

In [15]:
model.train(review_text, total_examples=model.corpus_count, epochs = model.epochs)

(61508644, 83868975)

In [16]:
model.corpus_count

194439

# Save the Model

In [17]:
model.save("./final_word2vec.model")

# Find the Similar Words and Similarity between the words

In [18]:
# Word Embeddings
model.wv.most_similar("bad")

[('terrible', 0.767177164554596),
 ('horrible', 0.7197291851043701),
 ('bleached', 0.6733271479606628),
 ('good', 0.6616243124008179),
 ('okay', 0.6594180464744568),
 ('anyways', 0.6535085439682007),
 ('buks', 0.653091311454773),
 ('qaulity', 0.6519861817359924),
 ('goodthe', 0.6493051052093506),
 ('soso', 0.6489846706390381)]

In [19]:
model.wv.similarity(w1='cheap',w2 = 'inexpensive')

0.659655

In [20]:
matrix = model.wv.vectors # extract the word embedding matrix

print(f"shape of the word embedding matrix is {matrix.shape}")

shape of the word embedding matrix is (35561, 100)


In [24]:
W_out = model.syn1neg
W_out.shape # Output weights

(35561, 100)

In [28]:
#https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4
import random
random_word = random.choice(model.wv.index_to_key)
random_word

'vac'

In [30]:
vocab_list = list(model.wv.index_to_key)

In [31]:
len(vocab_list)

35561

In [ ]:
vocab_list

In [47]:
model.wv

In [48]:
vocab_len = len(vocab_list)

In [ ]:
vocab['sum'].index

4697

In [ ]:
model.wv.index_to_key

In [ ]:
model.wv.key_to_index

In [49]:
vocab_int2word = {model.wv.key_to_index[w]:model.wv.get_vector(w, norm=True) for w in vocab_list}

In [ ]:
vocab_int2word

In [ ]:
vocab_int2word[4697]

In [55]:
import numpy as np
one_hot = np.zeros(shape=(1,vocab_len))

one_hot[0,model.wv.key_to_index['awesome']] = 1

In [56]:
z_proj = np.dot(one_hot,matrix)
z_proj.shape

(1, 100)

In [57]:
a_proj = z_proj

z_out = np.dot(a_proj,W_out.T)
z_out.shape

(1, 35561)

In [58]:

#Softmax
a_out = np.exp(z_out)/np.sum(np.exp(z_out))

In [59]:
a_out.shape

(1, 35561)

In [61]:
a_out = a_out.flatten()

indices = np.argsort(a_out) #sort in ascending order
indices = indices[::-1] # sort in descending order

for i in range(10):
  word_index = indices[i]
  word = model.wv.index_to_key[word_index]
  a_out_i = a_out[word_index]

  print(f"{word} with score {a_out_i}")

speaker with score 0.0005313869991126974
love with score 0.00046666584488679736
sound with score 0.0004475489166895298
looks with score 0.0004367667764883044
absolutely with score 0.00042941429329589493
an with score 0.0004112515980884151
awesome with score 0.00040937224080939117
fast with score 0.00040196066209755217
really with score 0.0003971751124775227
super with score 0.00039544769975203024
